In [ ]:
# 使用非公开数据进行模型效果的验证

import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import torch

# 使用cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(0)
torch.backends.cudnn.benchmark = True

unetweightspath="modelpths/unet9.pth"                           # UNet模型权重路径
truenoduleweightspath="modelpths/truenodule-cnn.pth"            # 用于判断是否是真结节的CNN模型权重路径
noduleclassweightspath="modelpths/NoduleCancerClassifier.pth"   # 用于判断结节癌性的CNN模型权重路径

In [ ]:
# 展示DICOM数据
from ProcessCTData import load_scan, get_pixels_hu, Standardizeimage

datafolder = './HPData/'
patients = os.listdir(datafolder)

def plot_2d(slices):
    fig, ax = plt.subplots(10, 4, figsize=(16, 40))
    for i in range(10,50):
        ax[(i-10)//4, (i-10)%4].imshow(slices[i])
        ax.flat[(i-10)].axis('off')
    plt.show()

def process_img(path):
    slices = load_scan(path)
    slices = get_pixels_hu(slices)
    plot_2d(slices)
    # print('原始3D图像, 形状：', slices.shape)
    # plot_3d(slices, 400)
    for i in range(len(slices)):
        slices[i] = Standardizeimage(slices[i])
    return slices

print(' ')
NoduleSlices = process_img(datafolder + ' ')
plot_2d(NoduleSlices)
# NoduleSlices = NoduleSlices.reshape(NoduleSlices.shape[0], 1, 512, 512)
# print('经处理后的 图像, 形状：', NoduleSlices.shape)
# plot_3d(NoduleSlices, 0)

print(' ')
CancerSlices = process_img(datafolder + ' ')
plot_2d(CancerSlices)
# CancerSlices = CancerSlices.reshape(CancerSlices.shape[0], 1, 512, 512)
# print('经处理后的 图像, 形状：', CancerSlices.shape)
# plot_3d(CancerSlices, 0)

del NoduleSlices, CancerSlices

In [ ]:
# 加载所需模型

# 导入肺结节分割模型
from NoduleUNet import UNet

unetmodel = UNet().to(device)
unetmodel.load_state_dict(torch.load(unetweightspath))
unetmodel.eval()

# 导入判断是否是真结节的模型
from TrueNoduleClassifier import NoduleClassifier

truenodulemodel = NoduleClassifier().to(device)
truenodulemodel.load_state_dict(torch.load(truenoduleweightspath))
truenodulemodel.eval()

# 导入判断结节癌性的模型    
from NoduleCancerClassifier import CNNModel# NoduleCancerClassifier

noduleclassmodel = CNNModel(2, 32).to(device)
noduleclassmodel.load_state_dict(torch.load(noduleclassweightspath))
noduleclassmodel.eval()


In [ ]:
# 使用UNet从原始图像中提取结节

# 预测图像的掩码
def predict_mask(images):
    num_test = images.shape[0]
    imgs_mask_test = np.ndarray([num_test, 1, 512, 512], dtype=np.float32)
    for i in range(num_test):
        imgs_mask_test[i] = unetmodel.forward(torch.from_numpy(images[i:i+1]).to(device).float()).cpu().detach().numpy()[0]
        torch.cuda.empty_cache()
    return imgs_mask_test

# 获取检测到结节的切片索引
def get_nodule_index(imgs_mask_test):
    mask_sum = [np.sum(maskslice[0]) for maskslice in imgs_mask_test]
    print('mask_sum: ', mask_sum)
    return [i for i in range(len(mask_sum)) if mask_sum[i] > 5]

# 获取真正的索引
def get_true_indices(processed_pix, nodule_index):
    nodule_imgs = [processed_pix[ind] for ind in nodule_index]
    nodule_imgs = np.array(nodule_imgs, np.float32)
    print('nodule_imgs shape: ', nodule_imgs.shape)
    outputs = truenodulemodel.forward(torch.from_numpy(nodule_imgs).to(device).float())
    del nodule_imgs
    torch.cuda.empty_cache()
    print('outputs: ', outputs)
    _, predictions = torch.max(outputs, dim=1)
    del outputs
    print('predictions: ', predictions)
    true_indices = [ind for i, ind in enumerate(nodule_index) if predictions[i] == 1]
    if len(true_indices) == 0:
        return nodule_index
    return true_indices


In [ ]:
# 从原始图像中提取结节
from ProcessCTData import process_image_from_file

nodule_images = []
nodule_masks = []
sample = []
area = []
nodule_indices = []

for i in range(len(patients)):
    print("Processing patient #", i)
    patient_scan = load_scan(datafolder + patients[i])
    patient_pix = get_pixels_hu(patient_scan)
    processed_pix = process_image_from_file(patient_pix)
    processed_pix[processed_pix==-0] = 0
    mask = predict_mask(processed_pix)
    nodule_index = get_nodule_index(mask)
    print("Nodule index: ", nodule_index)
    if len(nodule_index) == 0:
        continue
    
    true_inds = get_true_indices(processed_pix, nodule_index)
    print("True indices: ", true_inds)
    if len(true_inds) == 0:
        continue
    
    for ind in true_inds:
        nodule_images.append(patient_pix[ind])
        nodule_masks.append(mask[ind])
        sample.append(patients[i])
        area.append(np.sum(mask[ind]))
        nodule_indices.append(ind)

nodule_images = np.array(nodule_images)
nodule_masks = np.array(nodule_masks)


In [ ]:
# 查看数据形状

print('nodule_images shape: ', nodule_images.shape)
print('nodule_masks shape: ', nodule_masks.shape)


In [ ]:
# 变换形状

nodule_images = nodule_images[:, np.newaxis, :, :]
print('nodule_images shape: ', nodule_images.shape)


In [ ]:
# 查看图像与掩模

fig, ax = plt.subplots(2, 7, figsize=(21, 8))

for i in range(7):
    ax[0, i].axis('off')
    ax[0, i].imshow(nodule_images[i][0], cmap='gray')
    ax[0, i].set_title("Nodule Image")
    ax[1, i].axis('off')
    ax[1, i].imshow(nodule_masks[i][0], cmap='gray')
    ax[1, i].set_title("Nodule Mask")

plt.show()


In [ ]:
# 生成特征表

meannodule_HU = []
nodule_count = []
largest_area_list = []
eccentricity_list = []
diam_list = []
diammajor_list = []
spiculation_list = []

from tqdm.notebook import tqdm
from scipy.ndimage import label
from ProcessCTData import get_largest_nodule_properties

for i in tqdm(range(nodule_masks.shape[0])):
    print("Processing nodule #", i)
    mask = nodule_masks[i, 0]
    mask[mask > 0.5] = 1
    mask[mask < 0.5] = 0
    meannodule_HU.append(np.sum(nodule_images[i, 0] * mask) / np.sum(mask))
    labeled_array, features = label(mask)
    nodule_count.append(features)
    area, eccentricity, diam, diammajor, spiculation = get_largest_nodule_properties(nodule_masks[i, 0])
    largest_area_list.append(area)
    eccentricity_list.append(eccentricity)
    diam_list.append(diam)
    diammajor_list.append(diammajor)
    spiculation_list.append(spiculation)
featurestable = pd.DataFrame({"Patient": sample, "NoduleIndex": nodule_indices, "Area": area, 
                        "MeanHU": meannodule_HU, "LargestNoduleArea": largest_area_list,
                        "Eccentricity": eccentricity_list, "Diameter": diam_list, 
                        "DiameterMajor": diammajor_list, "Spiculation": spiculation_list})


In [ ]:
# 获取64x64肺结节切片

from ProcessCTData import processimagenomask, crop_nodule, largestnodulearea, largestnodulecoordinates

noduleexists = []
nodulecrops = []

biggestnodulearea = []

for j in range(nodule_masks.shape[0]):
    biggestnodulearea.append(largestnodulearea(nodule_masks[j, 0], featurestable, j))

featurestable["LargestNoduleArea"] = pd.Series(biggestnodulearea)

for patient in tqdm(patients):
    print("Process patient ", patient)
    nodulearea = featurestable[["LargestNoduleArea"]].loc[featurestable["Patient"] == patient]

    if len(nodulearea) > 0:
        index = nodulearea.loc[nodulearea["LargestNoduleArea"] > 25].index #只选出面积大于25的结节，即直径约大于2.8mm的结节
        noduleexists.extend(index)
        for i in range(len(index)):
            nodcrop = crop_nodule(largestnodulecoordinates(nodule_masks[i, 0]), processimagenomask(nodule_images[i, 0]))

            if nodcrop.shape[0] * nodcrop.shape[1] < 64 ** 2:
                nodulecrops.append(np.zeros([64, 64]))
                nodulecrops[i][0:nodcrop.shape[0], 0:nodcrop.shape[1]] = nodcrop
            else:
                nodulecrops.append(nodcrop)

nodulecrops = np.array(nodulecrops)
print(nodulecrops.shape)


In [ ]:
# 变换形状

nodulecrops = nodulecrops[:, np.newaxis, :, :]
print(nodulecrops.shape)


In [ ]:
# 显示64x64肺结节切片

fig, ax = plt.subplots(1, 4, figsize=(20, 10))

for i in range(4):
    ax[i].axis('off')
    ax[i].imshow(nodulecrops[i][0])
    ax[i].set_title("Nodule Image")

plt.show()

In [ ]:
# 预测肺结节的癌性

nodulecrops = torch.from_numpy(nodulecrops).float()

noduleclassmodel.eval()
with torch.no_grad():
    outputs = noduleclassmodel(nodulecrops.cuda())
    _, predicted = torch.max(outputs, 1)
    fig, ax = plt.subplots(1, 4, figsize=(20, 5))

    for i, j in enumerate((0, 1, 2, 3)):
        ax[i].imshow(nodulecrops[j, 0])
        ax[i].axis("off")
        ax[i].set_title("Malignant: " + str(bool(predicted[j].item())))
    plt.show()
